In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
def get_ind_file(filetype):
    """
    Load and format the Ken French 30 Industry Portfolios files
    """
    known_types = ["returns", "nfirms", "size"]
    if filetype not in known_types:
        raise ValueError(f"filetype must be one of:{','.join(known_types)}")
    if filetype is "returns":
        name = "vw_rets"
        divisor = 100
    elif filetype is "nfirms":
        name = "nfirms"
        divisor = 1
    elif filetype is "size":
        name = "size"
        divisor = 1
                         
    ind = pd.read_csv(f"data/ind30_m_{name}.csv", header=0, index_col=0)/divisor
    ind.index = pd.to_datetime(ind.index, format="%Y%m").to_period('M')
    ind.columns = ind.columns.str.strip()
    return ind

<>:8: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:8: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-2-84787575f706>:8: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if filetype is "returns":
<ipython-input-2-84787575f706>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif filetype is "nfirms":
<ipython-input-2-84787575f706>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif filetype is "size":


In [3]:
data = get_ind_file("returns")

In [8]:
# mu
mu_sample = data.mean()
grand_mean = mu_sample.mean()
mu_shrink = 0.25 * grand_mean + 0.75 * mu_sample # 0.25, 0.5, 0.75
# trenor ratio
# factor models

In [11]:
# sigma
covmat_sample = data.cov()
cormat_sample = data.corr()
stdvec_sample = data.std()

# constant correlation
n = cormat_sample.shape[0]
const_corr = (cormat_sample.sum().sum() - n) / (n**2 - n)
cormat_cc = np.ones_like(cormat_sample) * const_corr
np.fill_diagonal(cormat_cc, 1)
stdmat_cc = np.zeros_like(cormat_sample)
np.fill_diagonal(stdmat_cc, stdvec_sample)
covmat_cc = stdmat_cc @ cormat_cc @ stdmat_cc

covmat_shrink = 0.25 * covmat_cc + 0.75 * cormat_sample #缩水
# factor models

In [17]:
# EW
k_assets = data.shape[1]
weight_EW = np.array([1/k_assets]*k_assets)
portfolio_EW = data @ weight_EW

In [22]:
portfolio_EW

1926-07    0.026300
1926-08    0.035210
1926-09    0.004690
1926-10   -0.028967
1926-11    0.032877
             ...   
2018-08    0.009337
2018-09   -0.001257
2018-10   -0.074427
2018-11    0.018667
2018-12   -0.096270
Freq: M, Length: 1110, dtype: float64

In [26]:
cumret = (portfolio_EW + 1).cumprod()

runningmax = cumret.cummax()
max_drawdown = min(cumret/runningmax)-1

var95_sample = np.percentile(portfolio_EW, 5)
mu_EW = portfolio_EW.mean()
std_EW = portfolio_EW.std()
var95_standard = mu_EW - 2*std_EW

cvar95_sample = portfolio_EW[portfolio_EW <= var95_sample].mean()
cvar95_standard = portfolio_EW[portfolio_EW <= var95_standard].mean()

semi_deviation_zero = portfolio_EW[portfolio_EW <= 0].std()
semi_deviation_avg = portfolio_EW[portfolio_EW <= portfolio_EW.mean()].std()
semi_deviation_median = portfolio_EW[portfolio_EW <= portfolio_EW.median()].std()

In [23]:
from scipy.optimize import minimize

In [24]:
weight_EW.T @ mu_sample

0.010011756756756756

In [27]:
mu_EW

0.010011756756756737

In [29]:
con1 = {'type': 'ineq', 'fun': lambda w: np.sum(w)-1.0} # >= 0
con2 = {'type': 'ineq', 'fun': lambda w: 1.0-np.sum(w)} # >= 0
con3 = {'type': 'ineq', 'fun': lambda w: w} # >= 0

In [30]:
# MSR
# minimize minus sharpe ratio, W.T@mu /sqrt(W.T @ covmat @ W)
# subject to W.T @ W = 1
def _MSR_func(mu, covmat):
    def inner(W):
        return -(W.T @ mu) / (W.T @ covmat @ W)
    return inner

weight_MSR = minimize(_MSR_func(mu_sample, covmat_sample), # MSR function
                      weight_EW, # initial guess
                      method='SLSQP', # method
                      constraints=[con1,con2,con3] # constraints
                     ).x

In [33]:
mu_EW/std_EW

0.17954914874302688

In [34]:
def construct_portfolio(data, weight):
    portfolio = data @ weight
    mu = portfolio.mean()
    std = portfolio.std()
    sr = mu / std
    
    cumret = (portfolio + 1).cumprod()
    annualized_return = (cumret[-1]-cumret[0])*252/len(portfolio)
    annualized_std = std*np.sqrt(252/len(portfolio))
    
    runningmax = cumret.cummax()
    max_drawdown = min(cumret/runningmax)-1

    var95_sample = np.percentile(portfolio, 5)
    mu = portfolio.mean()
    std = portfolio.std()
    var95_standard = mu - 2*std

    cvar95_sample = portfolio[portfolio <= var95_sample].mean()
    cvar95_standard = portfolio[portfolio <= var95_standard].mean()

    semi_deviation_zero = portfolio[portfolio <= 0].std()
    semi_deviation_avg = portfolio[portfolio <= portfolio.mean()].std()
    semi_deviation_median = portfolio[portfolio <= portfolio.median()].std()
    metrics = {
            'mu':mu, 'std':std, 'sr':sr,
            'annualized_return':annualized_return, 'annualized_std':annualized_std,
            'max_drawdown':max_drawdown,
            'var95_sample':var95_sample, 'var95_standard':var95_standard,
            'cvar95_sample':cvar95_sample, 'cvar95_standard':cvar95_standard,
            'semi_deviation_zero':semi_deviation_zero, 'semi_deviation_avg':semi_deviation_avg, 
            'semi_deviation_median':semi_deviation_median
        }
    return portfolio, metrics, cumret

In [35]:
portfolio_MSR, metrics_MSR, _ = construct_portfolio(data, weight_MSR)

In [36]:
weight_GMV = minimize(_MSR_func(np.repeat(1, k_assets), covmat_sample), weight_EW, method='SLSQP', constraints=[con1,con2,con3]).x

In [37]:
portfolio_GMV, metrics_GMV, _ = construct_portfolio(data, weight_GMV)

In [40]:
# ERC
def calculate_portfolio_variance(covmat, W):
    return W.T @ covmat @ W

def calculate_risk_contribution(covmat, W):
    return (covmat @ W * W.T / calculate_portfolio_variance(covmat, W))

def _ERC_func(covmat, target_risk_contribution):
    def inner(W):
        risk_contribution = calculate_risk_contribution(covmat, W)
        return np.square(risk_contribution - target_risk_contribution).sum() 
    return inner

In [41]:
target_risk_contribution

NameError: name 'target_risk_contribution' is not defined

In [42]:
target_risk_contribution = weight_EW
weight_ERC = minimize(_ERC_func(covmat_sample, target_risk_contribution), weight_EW, method='SLSQP', constraints=[con1,con2,con3]).x

In [ ]:
calculate_risk_contribution(covmat_sample,weight_ERC)

In [ ]:
portfolio_ERC, metrics_ERC, _ = construct_portfolio(data, weight_ERC)

In [ ]:
metrics_MSR

In [ ]:
metrics_ERC

In [ ]:
# Black Litterman

In [ ]:
# factor model: mkt, size, value, momentum, vol; robust v. weak (operating profitability); aggressive ()

In [ ]:
# 1. 整理我写的 Weighting scheme
# 2. 整理你写的 Factor model -> cov matrix, expected returns
# 3. Black Litterman
# 4. OLS, ridge, lasso (一个值)
# 5. TimeSeries CV
# 6. 验证因子有效性：IC, IR, 分层backtest
# 7. 算出因子，存入database